## Homework 4: Neural Networks for Dynamical Systems
DUE: Wednesday, June 10, 2020

### Download the accompanying ZIP file which includes MATLAB code for solving (i) A reaction-diffusion system of equations, and (ii) The Kuramoto-Sivashinsky (KS) equation.

### 1. Train a NN that can advance the solution from t to t + ∆t for the KS equation


Define KS equation

Define Stochastic Gradient Descent Neural Network

Advance the solution

### 2. Compare your evolution trajectories for your NN against using the ODE time-stepper provided with different initial conditions

Advance KS with ODE and compare to NN output

### 3. For the reaction-diffusion system, first project to a low-dimensional subspace via the SVD and see how forecasting works in the low-rank variables.

Define reaction-diffusion system equation

SVD and forecast

Compare to ODE and NN outputs

### For the Lorenz equations (code given out previously in class emails), consider the following.
### 1. Train a NN to advance the solution from t to t+∆t for ρ=10, 28 and 40. Now see how well your NN works for future state prediction for ρ = 17 and ρ = 35.

Define Lorenz equations

Define Stochastic Gradient Descent Neural Network

Advance the solution

Evaluate future predictions

### 2. See if you can train your NN to identify (for ρ = 28) when a transition from one lobe to another is imminent. Determine how far in advance you can make this prediction. (NOTE: you will have to label the transitions in a test set in order to do this task)

## Scrap

In [2]:
# clear all; close all; clc

# % Kuramoto-Sivashinsky equation (from Trefethen)
# % u_t = -u*u_x - u_xx - u_xxxx,  periodic BCs 

# N = 1024;
# x = 32*pi*(1:N)'/N;
# u = cos(x/16).*(1+sin(x/16)); 
# v = fft(u);

# % % % % % %
# %Spatial grid and initial condition:
# h = 0.025;
# k = [0:N/2-1 0 -N/2+1:-1]'/16;
# L = k.^2 - k.^4;
# E = exp(h*L); E2 = exp(h*L/2);
# M = 16;
# r = exp(1i*pi*((1:M)-.5)/M);
# LR = h*L(:,ones(M,1)) + r(ones(N,1),:);
# Q = h*real(mean( (exp(LR/2)-1)./LR ,2)); 
# f1 = h*real(mean( (-4-LR+exp(LR).*(4-3*LR+LR.^2))./LR.^3 ,2)); 
# f2 = h*real(mean( (2+LR+exp(LR).*(-2+LR))./LR.^3 ,2));
# f3 = h*real(mean( (-4-3*LR-LR.^2+exp(LR).*(4-LR))./LR.^3 ,2));

# % Main time-stepping loop:
# uu = u; tt = 0;
# tmax = 100; nmax = round(tmax/h); nplt = floor((tmax/250)/h); g = -0.5i*k;
# for n = 1:nmax
# t = n*h;
# Nv = g.*fft(real(ifft(v)).^2);
# a = E2.*v + Q.*Nv;
# Na = g.*fft(real(ifft(a)).^2);
# b = E2.*v + Q.*Na;
# Nb = g.*fft(real(ifft(b)).^2);
# c = E2.*a + Q.*(2*Nb-Nv);
# Nc = g.*fft(real(ifft(c)).^2);
# v = E.*v + Nv.*f1 + 2*(Na+Nb).*f2 + Nc.*f3; if mod(n,nplt)==0
#         u = real(ifft(v));
# uu = [uu,u]; tt = [tt,t]; end
# end
# % Plot results:
# surf(tt,x,uu), shading interp, colormap(hot), axis tight 
# % view([-90 90]), colormap(autumn); 
# set(gca,'zlim',[-5 50]) 

# save('kuramoto_sivishinky.mat','x','tt','uu')

# %%
# figure(2), pcolor(x,tt,uu.'),shading interp, colormap(hot),axis off

In [4]:
# https://github.com/E-Renshaw/kuramoto-sivashinsky

In [5]:
# import numpy as np
# from mpl_toolkits.mplot3d import Axes3D
# import matplotlib.pyplot as plt
# from matplotlib import cm

# # KSequ.m - solution of Kuramoto-Sivashinsky equation
# #
# # u_t = -u*u_x - u_xx - u_xxxx, periodic boundary conditions on [0,32*pi]
# # computation is based on v = fft(u), so linear term is diagonal
# #
# # Using this program:
# # u is the initial condition
# # h is the time step
# # N is the number of points calculated along x
# # a is the max value in the initial condition
# # b is the min value in the initial condition
# # x is used when using a periodic boundary condition, to set up in terms of
# #   pi
# #
# # Initial condition and grid setup
# N = 1024
# x = np.transpose(np.conj(np.arange(1, N+1))) / N
# a = -1
# b = 1
# u = np.cos(x/16)*(1+np.sin(x/16))
# v = np.fft.fft(u)
# # scalars for ETDRK4
# h = 0.25
# k = np.transpose(np.conj(np.concatenate((np.arange(0, N/2), np.array([0]), np.arange(-N/2+1, 0))))) / 16
# L = k**2 - k**4
# E = np.exp(h*L)
# E_2 = np.exp(h*L/2)
# M = 16
# r = np.exp(1j*np.pi*(np.arange(1, M+1)-0.5) / M)
# LR = h*np.transpose(np.repeat([L], M, axis=0)) + np.repeat([r], N, axis=0)
# Q = h*np.real(np.mean((np.exp(LR/2)-1)/LR, axis=1))
# f1 = h*np.real(np.mean((-4-LR+np.exp(LR)*(4-3*LR+LR**2))/LR**3, axis=1))
# f2 = h*np.real(np.mean((2+LR+np.exp(LR)*(-2+LR))/LR**3, axis=1))
# f3 = h*np.real(np.mean((-4-3*LR-LR**2+np.exp(LR)*(4-LR))/LR**3, axis=1))
# # main loop
# uu = np.array([u])
# tt = 0
# tmax = 150
# nmax = round(tmax/h)
# nplt = int((tmax/100)/h)
# g = -0.5j*k
# for n in range(1, nmax+1):
#     t = n*h
#     Nv = g*np.fft.fft(np.real(np.fft.ifft(v))**2)
#     a = E_2*v + Q*Nv
#     Na = g*np.fft.fft(np.real(np.fft.ifft(a))**2)
#     b = E_2*v + Q*Na
#     Nb = g*np.fft.fft(np.real(np.fft.ifft(b))**2)
#     c = E_2*a + Q*(2*Nb-Nv)
#     Nc = g*np.fft.fft(np.real(np.fft.ifft(c))**2)
#     v = E*v + Nv*f1 + 2*(Na+Nb)*f2 + Nc*f3
#     if n%nplt == 0:
#         u = np.real(np.fft.ifft(v))
#         uu = np.append(uu, np.array([u]), axis=0)
#         tt = np.hstack((tt, t))
# # plot
# fig = plt.figure()
# ax = fig.gca(projection='3d')
# tt, x = np.meshgrid(tt, x)
# surf = ax.plot_surface(tt, x, uu.transpose(), cmap=cm.coolwarm, linewidth=0, antialiased=False)
# fig.colorbar(surf, shrink=0.5, aspect=5)
# plt.show()